In [23]:
# 必要なライブラリのインポート（アルファベット順に）
from bs4 import BeautifulSoup
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import requests

In [24]:
# アクセス先のUTLを指定
template_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?mt=9999999&cn=9999999&co=1&ta=13&url=/chintai/ichiran/FR301FC001/&et=9999999&smk=&shkr1=03&sc=13101&sc=13102&sc=13104&ar=030&bs=040&ct=20.0&shkr3=03&shkr2=03&mb=0&md=05&md=06&md=07&md=08&md=09&md=10&md=11&md=12&md=13&md=14&shkr4=03&cb=0.0&page={}"

In [25]:
# ページ数を取得
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?mt=9999999&cn=9999999&co=1&ta=13&url=/chintai/ichiran/FR301FC001/&et=9999999&smk=&shkr1=03&sc=13101&sc=13102&sc=13104&ar=030&bs=040&ct=20.0&shkr3=03&shkr2=03&mb=0&md=05&md=06&md=07&md=08&md=09&md=10&md=11&md=12&md=13&md=14&shkr4=03&cb=0.0"
res = requests.get(base_url)

# ステータスコードが正常なら実行
if res.status_code == 200: 

    html_data = res.text
    soup = BeautifulSoup(html_data, 'html.parser')

    page_elements = soup.select("ol.pagination-parts a")
    page_numbers = [int(element.text) for element in page_elements] # aタグ内に記載されたページ数をリスト化
    total_pages = max(page_numbers)

print("ページ数：" + str(total_pages))

ページ数：23


In [33]:
# データフレームを作成
columns = ["物件名","住所","間取り","階数","家賃","管理費","敷金","礼金","専有面積","築年数"]
df = pd.DataFrame(columns=columns)

df

,物件名,住所,間取り,階数,家賃,管理費,敷金,礼金,専有面積,築年数


In [34]:
# ステータスコードが正常なら実行
if res.status_code == 200: 
    html_data = res.text
    soup = BeautifulSoup(html_data, 'html.parser')

    # 各ページにアクセスして情報を取得
    for page_number in range(1, total_pages + 1):
        page_url = template_url.format(page_number)
        page_response = requests.get(page_url)

        # 必要な情報を抽出
        property_items = soup.select('.cassetteitem')

        for item in property_items:
            title = item.select_one('.cassetteitem_content-title').text  # 物件名の取得
            address = item.select_one('.cassetteitem_detail-col1').text  # 住所の取得
            age = item.select_one('.cassetteitem_detail-col3 div').text  # 築年数の取得
            
            floor_level = item.select_one('.js-cassette_link td:nth-of-type(3)').text.strip()  # 階数の取得
            rent_fee = item.select_one('.cassetteitem_price--rent .ui-text--bold').text  # 家賃の取得
            management_fee = item.select_one('.cassetteitem_price--administration').text  # 管理費の取得
            deposit = item.select_one('.cassetteitem_price--deposit').text  # 敷金の取得
            gratuity_fee = item.select_one('.cassetteitem_price--gratuity').text  # 礼金の取得
            occupied_area = item.select_one('.cassetteitem_menseki').text  # 専有面積の取得
            floor_plan = item.select_one('.cassetteitem_madori').text  # 間取りの取得

            # 抽出した情報をデータフレームに追加
            add_data = {
                "物件名": [title],
                "住所": [address],
                "間取り": [floor_plan],
                "階数": [floor_level],
                "家賃": [rent_fee],
                "管理費": [management_fee],
                "敷金": [deposit],
                "礼金": [gratuity_fee],
                "専有面積": [occupied_area],
                "築年数": [age]
            }

            add_df = pd.DataFrame(add_data)
            df = pd.concat([df, add_df], ignore_index=True)


In [35]:
df

,物件名,住所,間取り,階数,家賃,管理費,敷金,礼金,専有面積,築年数
0,東京メトロ丸ノ内線 四谷三丁目駅 10階建 築4年,東京都新宿区大京町,21K,2階,13.15万円,10000円,-,-,25.59m2,築4年
1,ＪＲ中央線 東中野駅 3階建 築33年,東京都新宿区北新宿３,4SK,1階,19万円,10000円,-,19万円,80m2,築33年
2,HF西新宿レジデンスWEST,東京都新宿区西新宿８,11DK,11階,13.6万円,12000円,13.6万円,-,32.64m2,築18年
3,ベター・ディズ,東京都新宿区北新宿１,14K,2階,7.7万円,4000円,-,-,16.95m2,築22年
4,松濤マンション,東京都新宿区北新宿３,4SK,1階,19万円,10000円,-,-,80m2,築21年
...,...,...,...,...,...,...,...,...,...,...
685,ソフトタウン晴海,東京都中央区晴海２,3DK,5階,15万円,10000円,15万円,15万円,54.88m2,築41年
686,高田馬場住宅,東京都新宿区下落合２,2DK,4階,16.3万円,5000円,16.3万円,16.3万円,45.84m2,築55年
687,新宿六丁目貸一戸建,東京都新宿区新宿６,4K,2階,17万円,-,34万円,17万円,58.66m2,築51年
688,高田馬場第二ロイヤルコーポ,東京都新宿区高田馬場４,2DK,3階,11万円,5000円,11万円,11万円,30.51m2,築54年


In [29]:
# Googleスプレッドシートの認証情報を取得
SCOPES = ["https://spreadsheets.google.com/feeds","https://www.googleapis.com/auth/drive"]
SERVICE_ACCOUNT_FILE = "tech0-step3_1_1-suumo-jsonkey.json"

credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE,SCOPES)
gs = gspread.authorize(credentials)

SPREADSHEET_KEY = "1rRtCz79tHmcBlmRGqbQDgwxgA_P8822bFhcb-k6MxQs"

workbook = gs.open_by_key(SPREADSHEET_KEY)
worksheet = workbook.worksheet("20231203")

In [30]:
# データフレームをGoogleスプレッドシートに転記
set_with_dataframe(worksheet,df,include_index=True)